In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ziadadelkhalifa","key":"377429fc9f6e3877774c2ceff6182ded"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
#Downloading the data from kaggle

!kaggle datasets download -d haithemhermessi/sanad-dataset

Dataset URL: https://www.kaggle.com/datasets/haithemhermessi/sanad-dataset
License(s): U.S. Government Works
 72% 47.0M/65.7M [00:00<00:00, 108MB/s]
100% 65.7M/65.7M [00:00<00:00, 111MB/s]


In [4]:
!unzip /content/sanad-dataset.zip -d text-data

Streaming output truncated to the last 5000 lines.
  inflating: text-data/Tech/1500.txt  
  inflating: text-data/Tech/1501.txt  
  inflating: text-data/Tech/1502.txt  
  inflating: text-data/Tech/1503.txt  
  inflating: text-data/Tech/1504.txt  
  inflating: text-data/Tech/1505.txt  
  inflating: text-data/Tech/1506.txt  
  inflating: text-data/Tech/1507.txt  
  inflating: text-data/Tech/1508.txt  
  inflating: text-data/Tech/1509.txt  
  inflating: text-data/Tech/1510.txt  
  inflating: text-data/Tech/1511.txt  
  inflating: text-data/Tech/1512.txt  
  inflating: text-data/Tech/1513.txt  
  inflating: text-data/Tech/1514.txt  
  inflating: text-data/Tech/1515.txt  
  inflating: text-data/Tech/1516.txt  
  inflating: text-data/Tech/1517.txt  
  inflating: text-data/Tech/1518.txt  
  inflating: text-data/Tech/1519.txt  
  inflating: text-data/Tech/1520.txt  
  inflating: text-data/Tech/1521.txt  
  inflating: text-data/Tech/1522.txt  
  inflating: text-data/Tech/1523.txt  
  inflating: 

In [5]:
#Collect all the text files from all the folders and make them in one text file


import os

# Directory containing the folders with text files
base_directory = r'/content/text-data'

# Output file for combined content
output_file = os.path.join(base_directory, '/content/combined_text.txt')

# Open the output file in write mode
with open(output_file, 'w', encoding='utf-8') as outfile:
    # Loop through each folder in the base directory
    for folder_name in sorted(os.listdir(base_directory)):
        folder_path = os.path.join(base_directory, folder_name)

        # Check if the path is a directory
        if os.path.isdir(folder_path):
            # Write folder name as a section header
            outfile.write(f'\n\n=== Content from Folder: {folder_name} ===\n\n')

            # Loop through all text files in the folder
            for filename in sorted(os.listdir(folder_path)):
                if filename.endswith('.txt'):
                    file_path = os.path.join(folder_path, filename)
                    with open(file_path, 'r', encoding='utf-8') as infile:
                        # Write the content of each file to the output file
                        outfile.write(infile.read())
                        outfile.write("\n")  # Add a newline to separate file contents

print(f'All text files have been concatenated into {output_file}')


All text files have been concatenated into /content/combined_text.txt


In [9]:
!pip install sentence_transformers
!pip install langchain
!pip install langchain_community
!pip install chromadb

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [11]:
#Applaying chunks in the text file that contain all my data

# Path to the text file
text_file_path = r'/content/combined_text.txt'

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define a basic Document class with page_content and metadata
class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata if metadata is not None else {}



# Read the content of the text file
with open(text_file_path, 'r', encoding='utf-8') as file:
    text_content = file.read()

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3450, chunk_overlap=250)

# Create a list of Document objects with metadata
documents = [Document(page_content=text_content, metadata={"source": text_file_path})]

# Split the text into smaller chunks
split_documents = text_splitter.split_documents(documents)


print(len(split_documents))



40593


In [12]:
#Applaying arabic text model in my text chunks to extract the vector database Embedding to use it in RAG system

from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

class Embedding:
    def __init__(self):
        # Use a model designed for Arabic text embeddings
        self.model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

        # Alternatively, you can try 'asafaya/bert-base-arabic' for better Arabic-specific embeddings
        self.model = SentenceTransformer(self.model_name)

    def embed_documents(self, docs):
        embeddings = self.model.encode(docs)
        return embeddings.tolist()

    def embed_query(self, query):
        return self.model.encode(query).tolist()

embed_model = Embedding()

# Assuming split_documents is already defined and contains the documents you want to embed
# Example: split_documents = ["الزكاة هي فريضة", "الصدقة تطوع"]

# Create a HuggingFaceEmbeddings object for Chroma using the model name
hf_embeddings = HuggingFaceEmbeddings(model_name=embed_model.model_name)

# Create the vector database using the documents and embedding model
vector_database = Chroma.from_documents(documents=split_documents, embedding=hf_embeddings)

retriever = vector_database.as_retriever(search_type='similarity', search_kwargs={'k': 3})





Some examples for retuen the nearest text to a specific text

In [13]:
retrieved_docs = retriever.invoke(' أعلنت منظمة التجارة الخارجية اليابانية «جيترو» نجاح مشاركتها في الدورة ال36 لأسبوع جيتكس للتكنولوجيا الذي استضافته دبي في المركز التجاري العالمي الأسبوع الماضي، حيث قدمت اليابان مجموعة من الابتكارات العلمية الجديدة في جناحها الذي استقبل حوالي 25 ألف زائر، ومستثمر.وعرضت الشركات اليابانية في دورة جيتكس 2016، ابتكارات سبع شركات تكنولوجيا ناشئة في مجالات متعددة، أهمها الطباعة ثلاثية الأبعاد التي تعرض من خلال مختبرات «بونساي» الشهيرة، و«مارس إليكتريك»، إلى جانب حلول الشبكات التكنولوجية من خلال مشاركة أجهزة «سيريفو»، وأجهزة عرض الفيديو عالية الدقة وأجهزة القياس الرياضي')

print(retrieved_docs[0].page_content)

حشدت الفطيم للتكنولوجيا، كافة إمكاناتها للمشاركة بأفضل الابتكارات في معرض جايتكس للتكنولوجيا، والذي يعقد خلال الفترة 18 22 أكتوبر/ تشرين الأول الجاري .وتعاونت الفطيم للتكنولوجيا، الفائزة بجائزتين متعاقبتين العام الحالي أحدثهما أفضل مزود للأنظمة المتكاملة في المنطقة، مع مركز دبي التجاري العالمي لإعداد وعرض تكنولوجيا المنازل والمكاتب الذكية المبتكرة بجناح تم تخصيصه لهذه الغاية في جايتكس .وقال فينكات راجافان، مدير عام الفطيم للتكنولوجيا: يسرنا رؤية مؤشرات تدل على تعافي القطاع، بعد عام اتسم بالصعوبة نوعاً ما، وهي أنباء جيدة لشركة الفطيم للتكنولوجيا وللجهات الفاعلة الرئيسية الإقليمية الأخرى .وأضاف: شكلنا جزءاً من معرض جايتكس لما يزيد على عقد من الزمن، إلا أن مشاركتنا للعام الجاري تعد خاصة بالفعل . ونشارك في المعرض وبحوزتنا جائزتان إقليميتان، وتتملكنا الحماسة لمواجهة المزيد من التحديات .وأشار إلى أنه سيتم عرض مهاراتنا المتخصصة وحلولنا في جناح المنازل والمكاتب الذكية، ونتقدم بالشكر للمنظمين لاختيارنا كجزء من هذه الفعالية . وسنقوم بعرض قدراتنا الفريدة من خلال الجمع بين مكونات متعددة، مع تركيز 

In [14]:
retrieved_docs = retriever.invoke('ستأصل الفريق الطبي الجراحي في مستشفى خليفة العام في أم القيوين حويصلة مرارية لمواطنة في العقد الرابع من العمر تحتوي على أكثر من 1600 حصوة ذات أحجام مختلفة.وأوضح رئيس الفريق الطبي استشاري الجراحة العامة وجراحة المناظير الدكتور طاهر حلمي، أن المريضة حضرت إلى العيادات الخارجية في المستشفى تعاني آلاماً مزمنة بالجانب العلوي الأيمن من البطن لأكثر من عام، حيث تم إجراء الأشعة فوق الصوتية والتي أظهرت عدداً كبيراً جداً من الحصوات بالحويصلة المرارية وقرر الفريق المشرف على الحالة بمشاركة الدكتورة صباح رمضاني أخصائية الجراحة العامة إجراء استئصال للحويصلة المرارية وما بها من حصوات، وذلك باستخدام المنظار الجراحي.وأضاف أنه بعد موافقة المريضة تم إجراء العملية واستغرقت قرابة ساعة وكانت المفاجأة أن عدد الحصوات يزيد على 1600 حصوة بمختلف الأحجام، حيث تم إجراء استئصال لها بنجاح ودون أية مضاعفات وأقامت المريضة في المستشفى لمدة يومين بعد العملية، حيث استقرت حالتها وغادرت وهي بصحة جيدة.وذكر أن هناك عدة عوامل لتشكل الحصوات المرارية وأهمها السمنة وتناول الأطعمة عالية الدهون وارتفاع الكوليسترول وارتفاع السكري،')

print(retrieved_docs[1].page_content)

أجرى الأطباء في مستشفى «كليفلاند كلينك أبوظبي»، أحد مرافق الرعاية الصحية عالمية المستوى التابعة لشركة مبادلة للاستثمار، عمليات ل «بضع عضل البواب» بواسطة المنظار عبر الفم، وهي تقنية مبتكرة تشهدها المنطقة لأول مرة، وتسهم في الحد من أعراض كسل المعدة الذي يعد من أمراض الجهاز الهضمي المزمنة. شهد المستشفى خلال العام الحالي علاج 4 مواطنين باستخدام هذا الإجراء، الذي يتم بأقل قدر ممكن من التدخل الجراحي، إذ يتيح للأطباء قطع البواب، وهو الصمام العضلي الذي يتم من خلاله تفريغ محتويات المعدة، عبر الفم دون جراحة، وقد ابتكرت التقنية في اليابان، وتعتمد على استخدام أدوات تنظير متطورة، توفر رؤية عالية الوضوح، كما تقلل من خطر مشاكل الجراحة المفتوحة ومضاعفاتها. وقال الدكتور ماثيو كرو، رئيس معهد أمراض الجهاز الهضمي في المستشفى: يعد كسل المعدة، من الأمراض التي تعيق قدرة المعدة على القيام بوظائفها في هضم الطعام وتفريغه بالشكل المناسب، ويعاني مرضى هذه الحالة من بعض الأعراض، كالغثيان الشديد والتقيؤ وحرقة المعدة وفقدان الوزن الشديد والنفخة وارتداد الحمض، موضحاً أن السكري يعد أحد الأسباب الرئيسية لحدوث كسل المعدة

In [15]:
retrieved_docs = retriever.invoke('أنه من الناحية النفسية فإن أطفال الشوارع يصابون بالقلق والتبول اللاإرادي إلى جانب بعض الأمراض النفسية الأخرى مثل الاكتئاب والعصبية والحقد على المجتمع، لأنهم يشعرون بعدم الأمان والظلم، أما من الناحية الجسدية')

print(retrieved_docs[0].page_content)

بعضهم في مراكز إيواء وتعليمهم مهنا مختلفة أو إلحاقهم بمدارس مفتوحة بالإضافة إلى الكشف عن العوامل البيئية -والأسرية والاجتماعية- التي أدت إلى تنامي هذه الظاهرة.مرض العزلة الاجتماعيةالدكتور أحمد البحيري استشاري الطب النفسي بالأمانة العامة للصحة النفسية يشير الى أنه من الناحية النفسية فإن أطفال الشوارع يصابون بالقلق والتبول اللاإرادي إلى جانب بعض الأمراض النفسية الأخرى مثل الاكتئاب والعصبية والحقد على المجتمع، لأنهم يشعرون بعدم الأمان والظلم، أما من الناحية الجسدية فهؤلاء الأطفال معرضون لحوادث سيارات أو اغتصاب أو تعلم عادات سيئة مثل التسول والسرقة وإدمان المخدرات.وأكد د. البحيري أن أطفال الشوارع بالرغم من أنهم يعيشون في المدن الكبيرة إلا أن معظمهم مصابون بمرض العزلة الاجتماعية الذي يعد من أخطر الأمراض التي تهدد المجتمعات الحديثة في العالم، لأنه يؤدي إلى انحسار الشخص في نفسه ويشعر بأنه مهمش ولا قيمة له في الدنيا ويفقد التخطيط للمستقبل ويسير في الشوارع بلا هدف ويفقد علاقته بالمجتمع ويعيش في جماعات وعصابات تلتزم بأخلاقيات الشارع وبالتالي يشكل خطورة كبيرة على المجتمع.الحل كما يرى د. البحيري ي

In [16]:
retrieved_docs = retriever.invoke('قترب نادي مانشستر يونايتد من ضم مدافع نادي فولهام كريس سمولينغ (20 عاماً) مقابل 8 ملايين جنيه بعد موافقة إدارة النادي على بيعه نهاية الموسم الحالي، وكان اليونايتد قد استبق التوقيع مع اللاعب الإنجليزي بعد أن أبدى أرسنال اهتمامه بضمه وقال متحدث باسم اليونايتد: مانشستر يونايتد وافق على شروط انتقال كريس سمولينغ من فولهام ووافق كريس على الشروط الشخصية وسيلحق بالفريق في حملته لموسم 2010 -2011 وعليه سيبقى في فولهام حتى نهاية الموسم الحالي .وكان سمولينغ لاعبا في ميدستون يونايتد غير المحترف قبل أن ينتقل لفولهام في مارس/ آذار 2008 ولعب الشهر الماضي مع فولهام وهو لاعب في منتخب إنجلترا دون ال 21 وعلق فينغر الذي خسر السباق على خدمات المدافع بالقول: يبدو أن اليونايتد عرض مالا أكثر منا')
print(retrieved_docs[0].page_content)

اقترب نادي مانشستر يونايتد من ضم مدافع نادي فولهام كريس سمولينغ (20 عاماً) مقابل 8 ملايين جنيه بعد موافقة إدارة النادي على بيعه نهاية الموسم الحالي، وكان اليونايتد قد استبق التوقيع مع اللاعب الإنجليزي بعد أن أبدى أرسنال اهتمامه بضمه وقال متحدث باسم اليونايتد: مانشستر يونايتد وافق على شروط انتقال كريس سمولينغ من فولهام ووافق كريس على الشروط الشخصية وسيلحق بالفريق في حملته لموسم 2010 -2011 وعليه سيبقى في فولهام حتى نهاية الموسم الحالي .وكان سمولينغ لاعبا في ميدستون يونايتد غير المحترف قبل أن ينتقل لفولهام في مارس/ آذار 2008 ولعب الشهر الماضي مع فولهام وهو لاعب في منتخب إنجلترا دون ال 21 وعلق فينغر الذي خسر السباق على خدمات المدافع بالقول: يبدو أن اليونايتد عرض مالا أكثر منا .وجاء عرض اليونايتد بعد الإصابات المتكررة التي تعرض لها مدافعه ريو فيرديناند علاوة على إمكانية رحيل فيديتش صوب ريال مدريد أو برشلونة . وعلق روي هودجسون مدير فولهام على الأنباء بالقول: النادي وافق على عرض من اليونايتد وليس لدي علم بوجود عرض من أرسنال . سير أليكس فيرجسون لا يخطئ في نظرته للمواهب الشابة ويدرك أنه بدفعه م

In [24]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the prompt template with a focus on context in Arabic
template = '''
أنت بوت ذكاء اصطناعي عربي تم تطويره من قبل البشر، ومهمتك هي الإجابة على أسئلة المستخدمين بناءً على المعرفة المقدمة في السياق أدناه.
السؤال سيكون باللغة العربية ويجب أن تكون الإجابة باللغة العربية أيضًا.
السياق المقدم لك يحتوي على سبع فئات: [الثقافة، المالية، الطب، السياسة، الدين، الرياضة، والتقنية].
حاول الإجابة باستخدام المعلومات المقدمة في السياق أدناه. إذا لم يكن الجواب موجودًا بوضوح في السياق، اتبع هذه الخطوات:
    1. اعتذر للمستخدم.
    2. أبلغ المستخدم أنك لا تعرف إجابة السؤال المطروح.
    3. أذكر أنك متخصص فقط في نماذج اللغة الكبيرة (LLMs).
    4. اسأل المستخدم إذا كان لديه المزيد من الأسئلة.
في نهاية الإجابة، اشكر المستخدم واسأله إذا كان لديه أي أسئلة أخرى.
يجب أن تكون الإجابة مفصلة ولا تقل عن 300 كلمة.
إذا سئلت عن نفسك، أخبر المستخدم أنك بوت دردشة تم تطويره من قبل البشر وأنك لست إنسانًا.

المعرفة التي تعرفها:
{context}

السؤال: {question}

الإجابة:
'''

def format_docs(docs):
    return "\n\n".join(docs)  # Assuming 'docs' is a list of strings

# Initialize the AraGPT2 model
aragpt2_model_name = "aubmindlab/aragpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(aragpt2_model_name)
aragpt2_model = GPT2LMHeadModel.from_pretrained(aragpt2_model_name)

# Function to generate a response using AraGPT2
def generate_aragpt2_response(context, question):
    input_text = f"{context}\n\nالسؤال: {question}\nالإجابة:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = aragpt2_model.generate(input_ids, max_length=500, num_return_sequences=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Create a custom RAG prompt
custom_rag_prompt = PromptTemplate.from_template(template)

# Define a dummy RAG chain for demonstration
def rag_chain(context, question):
    formatted_context = format_docs([context])  # Manually format the context
    answer = generate_aragpt2_response(formatted_context, question)
    return answer

# Example context to test
example_context = "هنا يوجد مثال على السياق الذي يحتوي على معلومات ذات صلة بسؤال المستخدم."

# Question to ask
question = 'اقترب نادي مانشستر يونايتد من ضم مدافع نادي فولهام كريس سمولينغ (20 عاماً) مقابل 8 ملايين جنيه بعد موافقة إدارة النادي على بيعه نهاية الموسم الحالي.'

# Generate the final answer using AraGPT2
answer = rag_chain(example_context, question)
print(answer)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


هنا يوجد مثال على السياق الذي يحتوي على معلومات ذات صلة بسؤال المستخدم.

السؤال: اقترب نادي مانشستر يونايتد من ضم مدافع نادي فولهام كريس سمولينغ (20 عاماً) مقابل 8 ملايين جنيه بعد موافقة إدارة النادي على بيعه نهاية الموسم الحالي.
الإجابة: بعد أن كان قد انتقل إلى نادي مانشستر يونايتد في صيف عام 2007 ، أصبح اللاعب الإنجليزي الشاب ( 20 عاما ) ، الذي يلعب في مركز الجناح الأيمن ، أحد أبرز اللاعبين في الدوري الإنجليزي الممتاز ، حيث لعب في صفوف الفريق الرديف ، كما لعب في صفوف الفريق الرديف أيضا.. بينما لعب سمولينغ في صفوف نادي فولهام ، الذي يلعب في دوري الدرجة الأولى الإنجليزي ، في موسم 2007 - 2008 ، حيث لعب في صفوفه حتى الآن ، حيث لعب في صفوف الفريق الرديف ، كما لعب في صفوف الفريق الرديف أيضا.. في حين لعب سمولينغ في صفوف نادي مانشستر يونايتد ، الذي يلعب في دوري الدرجة الأولى الإنجليزي ، في موسم 2008 - 2009 ، حيث لعب في صفوفه حتى الآن ، حيث لعب في صفوف الفريق الرديف ، كما لعب في صفوف الفريق الرديف أيضا.. في حين لعب سمولينغ في صفوف نادي مانشستر يونايتد ، الذي يلعب في دوري الدرجة الأولى الإنجلي

In [25]:
question ='استعادت جدّة إماراتية في الثانية والتسعين من العمر عافيتها، وكسبت فرصة جديدة للعيش، بعدما نجح الأطباء في العاصمة الإماراتية في إجراء عملية جراحية طارئة لإنقاذها من نوبة قلبية خطيرة ومضاعفات أخرى.'
# Generate the final answer using AraGPT2 with the formatted context and question
answer = rag_chain(example_context, question)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


هنا يوجد مثال على السياق الذي يحتوي على معلومات ذات صلة بسؤال المستخدم.

السؤال: استعادت جدّة إماراتية في الثانية والتسعين من العمر عافيتها، وكسبت فرصة جديدة للعيش، بعدما نجح الأطباء في العاصمة الإماراتية في إجراء عملية جراحية طارئة لإنقاذها من نوبة قلبية خطيرة ومضاعفات أخرى.
الإجابة: في عام 2006 ، كانت الطفلة التي تبلغ من العمر خمس سنوات ، قد خضعت لعملية جراحية في القلب ، حيث كانت تعاني من قصور في القلب ، مما أدى إلى توقف القلب عن العمل ، مما أدى إلى وفاتها........................................................................................................................................................................................................................................................................................................................................................................................................


In [34]:
question=' ؟عندما التقى حمد سعيد سلطان الشامسي سفير الدولة لدى بيروت، أمس، وائل أبو فاعور وزير الصحة اللبناني. وناقش الطرفان خلال اللقاء الذي عقد بمقر وزارة الصحة اللبنانية، ماذا ناقش'

answer = rag_chain(example_context, question)
print(answer)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


هنا يوجد مثال على السياق الذي يحتوي على معلومات ذات صلة بسؤال المستخدم.

السؤال:  ؟عندما التقى حمد سعيد سلطان الشامسي سفير الدولة لدى بيروت، أمس، وائل أبو فاعور وزير الصحة اللبناني. وناقش الطرفان خلال اللقاء الذي عقد بمقر وزارة الصحة اللبنانية، ماذا ناقش
الإجابة: م. محمد أبو العينين ، وزير الصحة الفلسطيني ، مع نظيره اللبناني ، الدكتور جواد خليفة ، حول سبل تعزيز التعاون الصحي بين البلدين.


**user friendly interface**

In [44]:
def generate_aragpt2_response(context, question):
    input_text = f"{context}\n\nالسؤال: {question}\nالإجابة:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = aragpt2_model.generate(input_ids, max_length=500, num_return_sequences=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

import gradio as gr

# Define the Gradio interface
def gradio_interface(context, question):
    return generate_aragpt2_response(context, question)

iface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(lines=5, placeholder="Enter the context here...", label="Context"),
        gr.Textbox(lines=1, placeholder="Enter your question here...", label="Question")
    ],
    outputs="text",
    title="Arabic Text Generation",
    description="Generate responses based on context and question using AraGPT2."
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e1379be4102abad88e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
